# Divisive Clustering

In [ ]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(font_scale=1.75)
sns.set_style("white")

np.random.seed(1)

Hierarchical clustering leads to a hierarchy of clusters. Two major types of hierarchical clustering algorithms are agglomerative and divisive. The former one starts with every data point in its own cluster and gradually merges cluters in a "bottom-up" fashion; the latter one assumes that all data points are in the same cluster initially and gradually divides it in a "top-down" fashion.

This DivisiveCluster algorithm implements hierarchical clustering in a “divisive” approach based on a chosen clustering algorithm such as [AutoGMM](./autogmm.ipynb). It retrieves predictions on the full dataset from the chosen clustering algorithm, say AutoGMM, and passes each subset of data corresponding to a predicted cluster onto AutoGMM again while specifying min_components=1. If the best model computed by AutoGMM for any predicted cluster leads to more than one subcluster, each of the subclusters will be clustered recursively as described above; otherwise, that subcluster becomes a leaf cluster. The algorithm terminates when all branches of recursive clustering have led to a set of leaf clusters.

## Using DivisiveCluster on Synthetic Data

Consider the following synthetic hierarchical data made up of two levels of four Gaussian distributions in 2D. The 4 distributions have the same standard deviation. And their means are chosen so that the difference between the first two is the same as that between the last two. Hence, this dataset can be classified into 4 clusters of 1 Gaussian component or 2 clusters of Gaussian mixtures of 2 components. Those are the two clustering hierarchies of increasing granularity.

In [ ]:
# generate synethetic data

np.random.seed(3)

n = 100  # number of data points
d = 2  # number of dimensions

# Let Xij denote the ith Gaussian mixture component in the jth cluster at the lowest hierarchy, i.e., level 2
X11 = np.random.normal(-4.5, 0.5, size=(n, d))
X21 = np.random.normal(-3, 0.5, size=(n, d))
X12 = np.random.normal(3, 0.5, size=(n, d))
X22 = np.random.normal(4.5, 0.5, size=(n, d))
X = np.vstack((X11, X21, X12, X22))

# true label at either level
y_lvl1 = np.repeat([0, 1], 2 * n).reshape((-1, 1))
y_lvl2 = np.repeat([0, 1, 2, 3], n).reshape((-1, 1))
y = np.hstack((y_lvl1, y_lvl2))

In [ ]:
# plotting function for clustering
def plot(X, y, title):
    df = pd.DataFrame(np.hstack((X,y)))
    df.columns = ["dim1", "dim2", "level1", "level2"]
    df["level1"] = df["level1"].astype(int)
    df["level2"] = df["level2"].astype(int)
    fig,ax = plt.subplots(1, figsize=(10,10))
    sns.scatterplot(
        data=df, x=df["dim1"], y=df["dim2"], style=df["level1"], hue=df["level2"], palette="deep", ax=ax, legend=False
    )
    ax.set(xticks=[], yticks=[], title=title)
    plt.show()

plot(X, y, "True Clustering")

In [ ]:
from graspologic.cluster import DivisiveCluster

# fit model and predict on data
dc = DivisiveCluster(max_components=2, cluster_method="gmm", max_level=2)
# choose to return a set of flat clusterings
pred = dc.fit_predict(X, fcluster=True)

### Visualize clustering assignments for both levels

In [ ]:
from graspologic.plot import pairplot_with_gmm

pairplot_with_gmm(X[np.argsort(pred[:,0])], dc.model_, title="Clustering Assignments at Level 1")
plt.show()

For level-2 assignments, we will make a Gaussian Mixture model which has 4 components, corresponding to the 4 leaf clusters.

In [ ]:
# function to reformat attributes of a Gaussian Mixture model such as "covarainces_"
def reformat_attr(gmm, gmm_attr, n_features):
    n_components = gmm.n_components
    if gmm.covariance_type == "tied":
        param = np.repeat(
            gmm_attr[np.newaxis, :, :], n_components, axis=0
        )
    elif gmm.covariance_type == "diag":
        param = np.array([
            np.diag(gmm_attr[i]) for i in range(n_components)
        ])
    elif gmm.covariance_type == "spherical":
        param = np.array([
            np.diag(np.repeat(gmm_attr[i], n_features))\
                for i in range(n_components)
        ])
    else:
        param = gmm_attr
        
    return param

In [ ]:
from sklearn.mixture import GaussianMixture

# collect key attributes of the leaf clusters
n_features = 2
n_clusters = 4
means = np.empty((1, n_features))
covs = np.empty((1, n_features, n_features))
prec_chols = np.empty((1, n_features, n_features))
for n_comp in range(n_clusters):
    means = np.vstack((means, dc.leaves[n_comp].model_.means_))
    cov = reformat_attr(dc.leaves[n_comp].model_, dc.leaves[n_comp].model_.covariances_, n_features)
    covs = np.vstack((covs, cov))
    prec_chol = reformat_attr(dc.leaves[n_comp].model_, dc.leaves[n_comp].model_.precisions_cholesky_, n_features)
    prec_chols = np.vstack((prec_chols, prec_chol))

means = means[1:, :]
covs = covs[1:, :]
prec_chols = prec_chols[1:, :]

# generate a new Gaussian Mixture model made up of 4 components
dc_gmm = GaussianMixture()
dc_gmm.means_ = means
dc_gmm.covariances_ = covs
dc_gmm.precisions_cholesky_ = prec_chols
dc_gmm.weights_ = np.repeat(1/n_clusters, n_clusters)
dc_gmm.n_components = n_clusters

In [ ]:
pairplot_with_gmm(X[np.argsort(pred[:,1])], dc_gmm, title="Clustering Assignments at Level 2")
plt.show()

### Evaluate clustering performance in terms of Adjusted Rand Index (ARI).

ARI is in the range of 0 to 1 where a score of 1 means the predicted assignments are identical to the true ones.

In [ ]:
from sklearn.metrics import adjusted_rand_score

print("ARI score for model at level 1: %.2f" % adjusted_rand_score(y[:,0], pred[:,0]))
print("ARI score for model at level 2: %.2f" % adjusted_rand_score(y[:,1], pred[:,1]))